In [2]:
from math import isclose
from itertools import product
import numpy as np
from numba import njit

In [3]:
def load_words(path="wordlist.txt"):
    with open(path, "r") as f:
        s = f.read()
    return s.split("\n")

def match(w1, w2):
    temp = ""
    ret = ""
    yellows = ""
    for i in range(5):
        char = w1[i]
        if char == w2[i]:
            temp += "G"
            yellows += char
        else:
            temp += "B"
    for i in range(5):
        char = w1[i]
        if temp[i] == "B" and w2.count(char) > yellows.count(char):
            ret += "Y"
            yellows += char
        else:
            ret += temp[i]
    return ret

In [4]:
@njit
def entropy(groups):
    groups = groups[groups != 0] / np.sum(groups)
    return -np.dot(groups, np.log2(groups))

In [5]:
def guess_wordle_information(guess="soare"):

    full_words = load_words("guesses.txt")
    words = load_words("possible_words.txt")
    conf = "guess"
    
    base_dict = {}
    acc = 0
    for x in product("GBY", repeat=5):
        base_dict["".join(x)] = acc
        acc += 1

    while True:

        print("the " + conf + " is", guess)
        if conf == "solution":
            return guess
        print("please enter wordle output from this")
        print("B for black, Y for yellow, G for green")
        truth = input().upper()

        if truth == "GGGGG":
            break
        assert len(truth) == 5
        for i in truth:
            assert i in "GYB"

        new_words = []
        yellows = ""
        for i in range(5):
            if truth[i] != "B":
                yellows += guess[i]

        for w in words:
            flag = True
            for i in range(5):
                char = guess[i]
                if truth[i] == "G" and w[i] != char:
                    flag = False
                    break
                elif truth[i] == "B" and (
                    w[i] == char or w.count(char) > yellows.count(char)
                ):
                    flag = False
                    break
                elif truth[i] == "Y" and (
                    w[i] == char or yellows.count(char) > w.count(char)
                ):
                    flag = False
                    break
            if flag:
                new_words.append(w)

        words = new_words
        guess = None
        avg_score = 0

        if len(words) == 1:
            guess = words[0]
            conf = "solution"
            continue
        
        print("possibilities:", len(words))

        for chance in full_words:
            groups = np.zeros(243)
            for answer in words:
                matching = match(chance, answer)
                groups[base_dict[matching]] += 1
            
            a = entropy(groups)
            
            if a > avg_score:
                avg_score = a
                guess = chance
            elif isclose(a, avg_score) and chance in words:
                guess = chance
                
        print("entropy", a)

    return guess

In [7]:
guess_wordle_information()

the guess is soare
please enter wordle output from this
B for black, Y for yellow, G for green


 gybbb


possibilities: 37
entropy 0.4804334574529622
the guess is cloot
please enter wordle output from this
B for black, Y for yellow, G for green


 ybbgb


the solution is scion


'scion'

In [11]:
def guess_wordle(guess="raise"):

    full_words = load_words("guesses.txt")
    words = load_words("possible_words.txt")
    conf = "guess"
    
    while True:

        print("the " + conf + " is", guess)
        if conf == "solution":
            return guess
        print("please enter wordle output from this")
        print("B for black, Y for yellow, G for green")
        truth = input().upper()

        if truth == "GGGGG":
            break
        assert len(truth) == 5
        for i in truth:
            assert i in "GYB"

        new_words = []
        yellows = ""
        for i in range(5):
            if truth[i] != "B":
                yellows += guess[i]

        for w in words:
            flag = True
            for i in range(5):
                char = guess[i]
                if truth[i] == "G" and w[i] != char:
                    flag = False
                    break
                elif truth[i] == "B" and (
                    w[i] == char or w.count(char) > yellows.count(char)
                ):
                    flag = False
                    break
                elif truth[i] == "Y" and (
                    w[i] == char or yellows.count(char) > w.count(char)
                ):
                    flag = False
                    break
            if flag:
                new_words.append(w)

        words = new_words
        guess = None
        avg_score = float("inf")

        if len(words) == 1:
            guess = words[0]
            conf = "solution"
            continue
        
        print("possibilities:", len(words))
        if len(words) == 2: print(words)

        for chance in full_words:
            groups = {}
            for answer in words:
                matching = match(chance, answer)
                if matching not in groups:
                    groups[matching] = 0
                groups[matching] += 1
            a = 0
            for i in groups.values():
                a += i ** 2
            if len(groups) == 0:
                guess = chance
                break
            a /= len(groups)
            if a < avg_score:
                avg_score = a
                guess = chance
            elif isclose(a, avg_score) and "GGGGG" in groups:
                guess = chance

    return guess